In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import numpy as np
import pandas as pd
from datetime import datetime
import logging
#from keras.layers import Dense
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
logger = tf.get_logger()
logging.basicConfig(
    format="%(asctime)s %(levelname)-8s %(message)s",
    datefmt="%Y-%m-%dT%H:%M:%SZ",
    level=logging.INFO)
print('Tensorflow-version: {0}'.format(tf.__version__))

Tensorflow-version: 2.2.0


In [2]:
data = pd.read_csv('s3://group2-code/heart.csv')

2021-03-17T15:47:38Z INFO     Found credentials in shared credentials file: ~/.aws/credentials


In [3]:
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
# Separate them to [X, Y]. Where our target labels are 'Y', and 'X' is our training data.
Y = data.target.values
X = data.drop(['target'], axis=1)

# Now split to train/test with 80% training data, and 20% test data.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Check dimensions of both sets.
print("Train Features Size:", X_train.shape)
print("Test Features Size:", X_test.shape)
print("Train Labels Size:", Y_train.shape)
print("Test Labels Size:", Y_test.shape)

Train Features Size: (242, 13)
Test Features Size: (61, 13)
Train Labels Size: (242,)
Test Labels Size: (61,)


In [5]:
# clear the logs
!rm -rf logs/

In [6]:
def NN_model(learning_rate):
    model = Sequential()
    model.add(Dense(32, input_dim=13, kernel_initializer='normal', activation='relu'))
    model.add(Dense(16, kernel_initializer='normal', activation='relu'))
    model.add(Dense(2, activation='softmax'))
    Adam(lr=learning_rate)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model

In [7]:
# Build a NN-model, and start training
learning_rate = 0.01
model = NN_model(learning_rate)
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                448       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 34        
Total params: 1,010
Trainable params: 1,010
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
def get_callbacks():
    # callbacks 
    # folder to store current training logs
    logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
    # print(logdir)
    
    class customLog(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            logging.info('epoch: {}'.format(epoch + 1))
            logging.info('loss={}'.format(logs['loss']))
            logging.info('accuracy={}'.format(logs['accuracy']))
            logging.info('val_accuracy={}'.format(logs['val_accuracy']))
    callbacks = [
        tf.keras.callbacks.TensorBoard(logdir),
        customLog()
    ]
    return callbacks

In [9]:
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=10, batch_size=16, verbose=2, callbacks=get_callbacks())

Epoch 1/10


2021-03-17T15:47:46Z INFO     epoch: 1
2021-03-17T15:47:46Z INFO     loss=1.2621265649795532
2021-03-17T15:47:46Z INFO     accuracy=0.5082644820213318
2021-03-17T15:47:46Z INFO     val_accuracy=0.44262295961380005


16/16 - 0s - loss: 1.2621 - accuracy: 0.5083 - val_loss: 0.6999 - val_accuracy: 0.4426


2021-03-17T15:47:46Z INFO     epoch: 2
2021-03-17T15:47:46Z INFO     loss=0.7032216787338257
2021-03-17T15:47:46Z INFO     accuracy=0.5289255976676941
2021-03-17T15:47:46Z INFO     val_accuracy=0.5245901346206665


Epoch 2/10
16/16 - 0s - loss: 0.7032 - accuracy: 0.5289 - val_loss: 0.6720 - val_accuracy: 0.5246
Epoch 3/10


...

16/16 - 0s - loss: 0.5772 - accuracy: 0.7149 - val_loss: 0.5040 - val_accuracy: 0.8361
Epoch 9/10


2021-03-17T15:47:48Z INFO     epoch: 9
2021-03-17T15:47:48Z INFO     loss=0.641739547252655
2021-03-17T15:47:48Z INFO     accuracy=0.5991735458374023
2021-03-17T15:47:48Z INFO     val_accuracy=0.6557376980781555


16/16 - 0s - loss: 0.6417 - accuracy: 0.5992 - val_loss: 0.5467 - val_accuracy: 0.6557


2021-03-17T15:47:48Z INFO     epoch: 10
2021-03-17T15:47:48Z INFO     loss=0.5640291571617126
2021-03-17T15:47:48Z INFO     accuracy=0.7190082669258118
2021-03-17T15:47:48Z INFO     val_accuracy=0.8852459192276001


Epoch 10/10
16/16 - 0s - loss: 0.5640 - accuracy: 0.7190 - val_loss: 0.4860 - val_accuracy: 0.8852


In [10]:
predictions = np.argmax(model.predict(X_test), axis=1)
model_accuracy = accuracy_score(Y_test, predictions)*100
print("Model Accracy:", model_accuracy,"%")
print(classification_report(Y_test, predictions))

Model Accracy: 88.52459016393442 %
              precision    recall  f1-score   support

           0       0.87      0.90      0.88        29
           1       0.90      0.88      0.89        32

    accuracy                           0.89        61
   macro avg       0.88      0.89      0.89        61
weighted avg       0.89      0.89      0.89        61



Tensorboard
Note : If you want to use Tensorboard : use tensorboard command and run from the terminal ( from notebook home page -> new -> terminal )

tensorboard --logdir=/home/jovyan/logs/ --bind_all
if you are running inside a container you can use port-mapping. if you are running inside kubernetes pod, then use the pod port-forward feature on the port 6006 (default for tensorboard, change it as per the tensorboard command output ). When a notebook is created, a pod with name <NOTEBOOK_NAME>-0 is created in the users namespace. So you can use the port-forward to access tensorboard.

kubectl port-forward -n <NAMESPACE> <NOTEBOOK_NAME>-0 6006:6006
kubectl -n bitspilanigroup2 port-forward my-nb-3-0 6006:6006

In [11]:
print('END OF NOTEBOOK')

END OF NOTEBOOK
